# **Fine-Tuning MPRNet:**

In this code, our goal is to use the pretrained weights as the initial values and fine-tune the entire network by training. This is time consuming, but the network will be trained for our custom dataset. The results show significant improvement of performance compared to the pretrained model:

**1)** We are using the best configs of google colab to run all of our codes. Therefore, the first step is to mount google drive, where we have uploaded the MPRNet github codes + our datasets + pretrained and best trained models:

In [ ]:
import os

#mount drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

!ls

/
Mounted at /content/gdrive
bin	 dev   lib32  mydrive			python-apt  srv    usr
boot	 etc   lib64  NGC-DL-CONTAINER-LICENSE	root	    sys    var
content  home  media  opt			run	    tmp
datalab  lib   mnt    proc			sbin	    tools


**2)** Make the MPRNet directory, the main working directory:

In [ ]:
os.chdir('mydrive/MPRNET_V2/Deblurring')
!ls

checkpoints	Datasets4		  test.py
checkpoints2	Datasets5		  training2.yml
CHK2		evaluate_GOPRO_HIDE.m	  training3.yml
CHK3		evaluate_RealBlur.py	  training4.yml
CHK4		Input			  training5.yml
CHK5		losses.py		  training6.yml
config.py	MPRNet.py		  training_Nov.yml
data_RGB.py	MPRNET_Training_V1.ipynb  training.yml
dataset_RGB.py	pretrained_models	  train.py
Datasets	__pycache__		  utils
Datasets2	README.md
Datasets3	Result


In [ ]:
os.getcwd()
!ls

checkpoints	Datasets4		  test.py
checkpoints2	Datasets5		  training2.yml
CHK2		evaluate_GOPRO_HIDE.m	  training3.yml
CHK3		evaluate_RealBlur.py	  training4.yml
CHK4		Input			  training5.yml
CHK5		losses.py		  training6.yml
config.py	MPRNet.py		  training_Nov.yml
data_RGB.py	MPRNET_Training_V1.ipynb  training.yml
dataset_RGB.py	pretrained_models	  train.py
Datasets	__pycache__		  utils
Datasets2	README.md
Datasets3	Result


**3)** Install required packages:

In [ ]:
!pip install yacs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install warmup-scheduler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for warmup-scheduler: filename=warmup_scheduler-0.3-py3-none-any.whl size=2983 sha256=a00b1878697e94f51862659b99dc1f9beedb86c5a9344d512dc2f478e0781473
  Stored in directory: /root/.cache/pip/wheels/f2/ce/4a/215c4f0add432420ff90fe04656bf2664ddfac7302e2b6fe51
Successfully built warmup-scheduler


**4)** The .yml file includes the training hyperparameters (patch size, batch size, learning rate, etc.). Inaddition to those, the directories of the initial model and the dataset is also included.

In [ ]:
from config import Config 
opt = Config('training6.yml')

gpus = ','.join([str(i) for i in opt.GPU])
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = gpus

import torch
torch.backends.cudnn.benchmark = True

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import random
import time
import numpy as np

import utils
from data_RGB import get_training_data, get_validation_data
from MPRNet import MPRNet
import losses
from warmup_scheduler import GradualWarmupScheduler
from tqdm import tqdm
from pdb import set_trace as stx

**5)** As mentioned, the PNSR calculation we use exploits random patches from the image. To make this calculation repetitive, we set the random seeds to fixed values to avoid changes each time we run the code:

In [ ]:

######### Set Seeds ###########
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)
torch.cuda.manual_seed_all(1234)

start_epoch = 1
mode = opt.MODEL.MODE
session = opt.MODEL.SESSION

result_dir = os.path.join(opt.TRAINING.SAVE_DIR, mode, 'results', session)
model_dir  = os.path.join(opt.TRAINING.SAVE_DIR, mode, 'models',  session)

utils.mkdir(result_dir)
utils.mkdir(model_dir)

train_dir = opt.TRAINING.TRAIN_DIR
val_dir   = opt.TRAINING.VAL_DIR

In [ ]:
import cv2
import os

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            # image_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) 
            # images.append(image_rgb)\
            images.append(img)
    return images

**6)** Find the validation and training datasets directories based on the given ".yml" file, creating the dataloaders, and definding the model + optimizer.

In [ ]:
# %%capture

######### Model ###########
model_restoration = MPRNet()
model_restoration.cuda()

device_ids = [i for i in range(torch.cuda.device_count())]
if torch.cuda.device_count() > 1:
  print("\n\nLet's use", torch.cuda.device_count(), "GPUs!\n\n")


new_lr = opt.OPTIM.LR_INITIAL

optimizer = optim.Adam(model_restoration.parameters(), lr=new_lr, betas=(0.9, 0.999),eps=1e-8)


######### Scheduler ###########
warmup_epochs = 3
scheduler_cosine = optim.lr_scheduler.CosineAnnealingLR(optimizer, opt.OPTIM.NUM_EPOCHS-warmup_epochs, eta_min=opt.OPTIM.LR_MIN)
scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=warmup_epochs, after_scheduler=scheduler_cosine)
scheduler.step()

######### Resume ###########
if opt.TRAINING.RESUME:
    path_chk_rest    = utils.get_last_path(model_dir, '_latest.pth')
    utils.load_checkpoint(model_restoration,path_chk_rest)
    # start_epoch = utils.load_start_epoch(path_chk_rest) + 1
    # utils.load_optim(optimizer, path_chk_rest)

    for i in range(1, start_epoch):
        scheduler.step()
    new_lr = scheduler.get_lr()[0]
    print('------------------------------------------------------------------------------')
    print("==> Resuming Training with learning rate:", new_lr)
    print('------------------------------------------------------------------------------')

if len(device_ids)>1:
    model_restoration = nn.DataParallel(model_restoration, device_ids = device_ids)

######### Loss ###########
criterion_char = losses.CharbonnierLoss()
criterion_edge = losses.EdgeLoss()

######### DataLoaders ###########
train_dataset = get_training_data(train_dir, {'patch_size':opt.TRAINING.TRAIN_PS})
train_loader = DataLoader(dataset=train_dataset, batch_size=opt.OPTIM.BATCH_SIZE, shuffle=True, num_workers=16, drop_last=False, pin_memory=True)

val_dataset = get_validation_data(val_dir, {'patch_size':opt.TRAINING.VAL_PS})
val_loader = DataLoader(dataset=val_dataset, batch_size=16, shuffle=False, num_workers=8, drop_last=False, pin_memory=True)

print('===> Start Epoch {} End Epoch {}'.format(start_epoch,opt.OPTIM.NUM_EPOCHS + 1))
print('===> Loading datasets')

/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


------------------------------------------------------------------------------
==> Resuming Training with learning rate: 3.3333333333333335e-05
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


===> Start Epoch 1 End Epoch 3001
===> Loading datasets


**7)** Train:

In [ ]:
# %%capture

best_psnr = 0
best_epoch = 0

for epoch in range(start_epoch, opt.OPTIM.NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    epoch_loss = 0
    train_id = 1

    model_restoration.train()
    for i, data in enumerate(tqdm(train_loader), 0):

        # zero_grad
        for param in model_restoration.parameters():
            param.grad = None

        target = data[0].cuda()
        input_ = data[1].cuda()

        restored = model_restoration(input_)
 
        # Compute loss at each stage
        loss_char = sum([criterion_char(restored[j],target) for j in range(len(restored))])
        # loss_char = np.sum([criterion_char(restored[j],target) for j in range(len(restored))])
        # loss_edge = np.sum([criterion_edge(restored[j],target) for j in range(len(restored))])
        loss_edge = sum([criterion_edge(restored[j],target) for j in range(len(restored))])
        loss = (loss_char) + (0.05*loss_edge)
       
        loss.backward()
        optimizer.step()
        epoch_loss +=loss.item()

    #### Evaluation ####
    if epoch%opt.TRAINING.VAL_AFTER_EVERY == 0:
        model_restoration.eval()
        psnr_val_rgb = []
        for ii, data_val in enumerate((val_loader), 0):
            target = data_val[0].cuda()
            input_ = data_val[1].cuda()

            with torch.no_grad():
                restored = model_restoration(input_)
            restored = restored[0]

            for res,tar in zip(restored,target):
                psnr_val_rgb.append(utils.torchPSNR(res, tar))

        psnr_val_rgb  = torch.stack(psnr_val_rgb).mean().item()

        if psnr_val_rgb > best_psnr:
            best_psnr = psnr_val_rgb
            best_epoch = epoch
            torch.save({'epoch': epoch, 
                        'state_dict': model_restoration.state_dict(),
                        'optimizer' : optimizer.state_dict()
                        }, os.path.join(model_dir,"model_best.pth"))

        print("[epoch %d PSNR: %.4f --- best_epoch %d Best_PSNR %.4f]" % (epoch, psnr_val_rgb, best_epoch, best_psnr))

        torch.save({'epoch': epoch, 
                    'state_dict': model_restoration.state_dict(),
                    'optimizer' : optimizer.state_dict()
                    }, os.path.join(model_dir,f"model_epoch_{epoch}.pth")) 

    scheduler.step()
    
    print("------------------------------------------------------------------")
    print("Epoch: {}\tTime: {:.4f}\tLoss: {:.4f}\tLearningRate {:.6f}".format(epoch, time.time()-epoch_start_time, epoch_loss, scheduler.get_lr()[0]))
    print("------------------------------------------------------------------")

    torch.save({'epoch': epoch, 
                'state_dict': model_restoration.state_dict(),
                'optimizer' : optimizer.state_dict()
                }, os.path.join(model_dir,"model_latest.pth")) 

100%|██████████| 350/350 [02:19<00:00,  2.51it/s]


------------------------------------------------------------------
Epoch: 1	Time: 139.5881	Loss: 17.2479	LearningRate 0.000067
------------------------------------------------------------------


100%|██████████| 350/350 [02:06<00:00,  2.77it/s]


[epoch 2 PSNR: 31.3730 --- best_epoch 2 Best_PSNR 31.3730]
------------------------------------------------------------------
Epoch: 2	Time: 146.6129	Loss: 16.3677	LearningRate 0.000100
------------------------------------------------------------------


100%|██████████| 350/350 [02:06<00:00,  2.77it/s]
/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:807: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


...

------------------------------------------------------------------
Epoch: 147	Time: 127.5614	Loss: 14.2957	LearningRate 0.000099
------------------------------------------------------------------


100%|██████████| 350/350 [02:07<00:00,  2.75it/s]


[epoch 148 PSNR: 31.9617 --- best_epoch 146 Best_PSNR 31.9929]
------------------------------------------------------------------
Epoch: 148	Time: 132.3032	Loss: 14.5988	LearningRate 0.000099
------------------------------------------------------------------


 22%|██▏       | 77/350 [00:29<01:43,  2.65it/s]


KeyboardInterrupt: ignored